## Checklist

### 진행예정
 1) 추가필요 열: 날짜 \
 2) 타 플랫폼과 합쳤을 때 형식 통일하도록 data type과 Nan 처리방법 등 정의 \
 3) 메리어트 계열사 제외처리 필요
 

### 완료
 1) page바꿔가면서 했을 때 상품들 연속성 CHK (limit=50에 따른 상품 누락 여부 확인) -> 이상없음. \
 2) page=1, 2 이상 구분할 때 [4], [1]이 아니라 '전체'를 가져오게 해야겠다. 시간 저녁되니까 숫자로 안된다. -> 해결 \
 3) today 기준시 기준으로, 한국보다 9시간 느림 -> 한국시로 맞추기 -> 해결
 4) 리뷰 추가 필요 -> 해결
 5) headers에서 app-version 나중에 업데이트 필요할 수도 있음 -> 업데이트 이전의 app-version 넣어도 이상없음 -> 해결
### 추후 체크 디버깅 예상
 1) X

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 1. 프로젝트 생성

In [4]:
# !rm -rf dailyhotel
!scrapy startproject dailyhotel

New Scrapy project 'dailyhotel', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/project/codes/crawling/dailyhotel

You can start your first spider with:
    cd dailyhotel
    scrapy genspider example example.com


In [3]:
!tree dailyhotel

dailyhotel
├── DailyhotelKeys.py
├── __pycache__
│   └── DailyhotelKeys.cpython-36.pyc
├── dailyhotel
│   ├── __init__.py
│   ├── __pycache__
│   │   ├── __init__.cpython-36.pyc
│   │   ├── items.cpython-36.pyc
│   │   ├── mongodb.cpython-36.pyc
│   │   ├── pipelines.cpython-36.pyc
│   │   └── settings.cpython-36.pyc
│   ├── items.py
│   ├── middlewares.py
│   ├── mongodb.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       ├── __pycache__
│       │   ├── __init__.cpython-36.pyc
│       │   └── spider.cpython-36.pyc
│       └── spider.py
├── dailyhotel.csv
└── scrapy.cfg

5 directories, 19 files


In [28]:
# 2. xpath 찾기: headers 설정

### hotel_key 가져오는 모듈 생성

In [10]:
# hotel key 얻는 모듈 생성 -> dailyhotel.py
def get_hotel_keys():
    import scrapy, requests
    from scrapy.http import TextResponse
    import datetime
    
    
    KST = datetime.timezone(datetime.timedelta(hours=9))
    today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
    start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
    end_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # 종료일 디폴트 = 내일
    staydays = (datetime.datetime.strptime(end_date,'%Y-%m-%d').date() - \
                datetime.datetime.strptime(start_date,'%Y-%m-%d').date()).days # 숙박일수 디폴트 = 1박start_date, end_date
    persons, staydays = 1, 1
    
    
    # 호텔리스트 가져올 parent_url 생성
    page = 1
    hotelidx = []
    while True:
        parent_url = f"https://www.dailyhotel.com/newdelhi/goodnight/api/v4/stays/sales?dateCheckIn={start_date}&regionIdx=30070&shortCutType=hotel_resort&persons={persons}&stays={staydays}&details=true&regionStayType=hotel_resort&stayType=hotel,resort&page={page}&limit=50"
        headers = {'app-version': '2.2.14'}
        req = requests.get(parent_url, headers=headers)
        response = req.json()
        list_all = response['data']['staySaleSections']

        if len(list_all) > 0:
            for each in list_all:
                if each['sectionViewType'] == 'NORMAL_섹션뷰타입':
                    lists = each['staySales']
                    print('page=', str(page))
                    page += 1
        else:
            print('All registered hotels have been searched.') # 이거는 확인용
            break


        # parent_url에서 각 호텔들의 Index number 가져오기
        for each in lists:
            hotelidx.append(each['hotelIdx'])
    
    return hotelidx

### 가져온 hotel_keys로 xpath찾기

In [22]:
# 일단 hotel_keys 받고 url 만들어서 yield시킬 예정
# 테스트
idx = hotelidx[0]
staydays = 1
start_date = "2021-03-10"


hotel_url = f'https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/{idx}?stays={staydays}&dateCheckIn={start_date}&regionStayType=all'
headers = {
    'app-version': '2.2.13'
}
req = requests.get(hotel_url, headers=headers)
response = TextResponse(req.url, body=req.text, encoding='utf-8')
resp_content = response.json()


#name
name = resp_content['data']['name'] 
#level
level = resp_content['data']['stayGradeName'] 
#score
try:
    score = round(resp_content['data']['rating']['values']/100*5, 1) 
except:
    score = '점수없음'
# location
location = resp_content['data']['address']
# roomtype
roomtype = []
for each in resp_content['data']['rooms']:
    roomtype.append(each['roomName'])
#price
price = []
for each in resp_content['data']['rooms']:
    price.append(each['amount']['discountAverage'])
#review
review = resp_content['data']['statistic']['reviewScoreTotalCount']
#link
hotel_desc_url = f"https://www.dailyhotel.com/stays/{idx}?dateCheckIn={start_date}&stays={staydays}" # 크롤링이랑 예약링크 다름 -> request url과 다르도록!

# print(name)
# print(level)
# print(score)
# print(location)
# print(roomtype)
# print(price)
# print(review)
# print(hotel_desc_url)

1491

In [12]:
# 3. item.py

In [4]:
%%writefile dailyhotel/dailyhotel/items.py
import scrapy


class DailyhotelItem(scrapy.Item):
    date = scrapy.Field()
    platform = scrapy.Field()
    name = scrapy.Field()
    level = scrapy.Field()
    score = scrapy.Field()
    review_count = scrapy.Field()
    location = scrapy.Field()
    room_type = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Overwriting dailyhotel/dailyhotel/items.py


# 4. spider.py

In [5]:
%%writefile dailyhotel/dailyhotel/spiders/spider.py
import scrapy, requests
import json
import datetime
from DailyhotelKeys import get_hotel_keys
from dailyhotel.items import DailyhotelItem


class DailyhotelSpider(scrapy.Spider):
    name = "Dailyhotel"
    allow_domain=["dailyhotel.com"]
    KST = datetime.timezone(datetime.timedelta(hours=9))
    today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
    start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
    end_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # 종료일 디폴트 = 내일
    staydays = (datetime.datetime.strptime(end_date,'%Y-%m-%d').date() - \
                datetime.datetime.strptime(start_date,'%Y-%m-%d').date()).days # 숙박일수 디폴트 = 1박start_date, end_date
    persons, staydays = 2, 1
    start_urls= ["https://www.dailyhotel.com/"]
    
    
    def parse(self, response):
        hotel_idx = get_hotel_keys()
        KST = datetime.timezone(datetime.timedelta(hours=9))
        today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
        start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘
        links = []
        start_urls = "https://www.dailyhotel.com/"
        headers = {'app-version': '2.2.14',
                   'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36',
                  }
        for idx in hotel_idx:
            hotel_url = start_urls + f'newdelhi/goodnight/api/v9/hotel/{idx}?stays=1&dateCheckIn={start_date}&regionStayType=all'
            links.append(hotel_url)   
        for link in links:
            yield scrapy.Request(link, headers=headers, callback=self.parse_content)
    
    
    def parse_content(self, response):
        KST = datetime.timezone(datetime.timedelta(hours=9))
        today = datetime.datetime.now(tz=KST) # 한국시간으로 설정
        start_date= today.strftime("%Y-%m-%d") # 시작일 디폴트 = 오늘  
        resp_content = response.json()
        staydays=1        
        #date
        date = start_date
        #name
        name = resp_content['data']['name'] 
        #level
        level = resp_content['data']['stayGradeName'] 
        #score
        try:
            score = float(round(resp_content['data']['rating']['values']/100*5, 1))
        except:
            score = float(0)
        # location
        location = resp_content['data']['address']
        # roomtype
        roomtypes = []
        for each in resp_content['data']['rooms']:
            roomtypes.append(each['roomName'])
        #price
        prices = []
        for each in resp_content['data']['rooms']:
            prices.append(int(each['amount']['discountAverage']))
        #review
        review = resp_content['data']['statistic']['reviewScoreTotalCount']
        #link
        hotel_id = resp_content['data']['idx']
        hotel_desc_url = f"https://www.dailyhotel.com/stays/{hotel_id}?dateCheckIn={start_date}&stays={staydays}" # 크롤링이랑 예약링크 다름 -> request url과 다르도록!
        #platform
        platform = "데일리호텔"

        
        item = DailyhotelItem()
        for i in range(len(roomtypes)):
            item['date'] = date
            item['platform'] = platform
            item['name'] = name
            item['level'] = level
            item['score'] = score
            item['review_count'] = review
            item['location'] = location
            item['room_type'] = roomtypes[i]
            item['price'] = prices[i]
            item['link'] = hotel_desc_url     
            yield item

Overwriting dailyhotel/dailyhotel/spiders/spider.py


In [30]:
# 5. settings.py: robots.txt 여부 파악

In [31]:
# 6. 프로젝트 실행

In [36]:
%pwd

'/home/ubuntu/python3/project/codes/crawling'

In [6]:
%%writefile run.sh
cd dailyhotel
rm dailyhotel.csv
scrapy crawl Dailyhotel -o dailyhotel.csv 

Overwriting run.sh


In [1]:
# 7. pipeline.py: 크롤링데이터 mongodb에 저장

In [2]:
# scrapy project에 pymongo module 생성

In [7]:
%%writefile dailyhotel/dailyhotel/mongodb.py
import pymongo


client = pymongo.MongoClient("mongodb://52.78.161.13:27017")
collection = client.hotel_hs.hotel

Overwriting dailyhotel/dailyhotel/mongodb.py


In [26]:
# pipeline.py 설정

In [8]:
%%writefile dailyhotel/dailyhotel/pipelines.py
from itemadapter import ItemAdapter
from .mongodb import collection

class DailyhotelPipeline:
    def process_item(self, item, spider):
        data = {"date": item['date'], "platform": item['platform'], "name": item['name'],
                "level": item['level'], "score": item['score'], "review": item['review_count'],
                "location": item['location'], "room_type": item['room_type'],
                "price": item['price'], "link": item['link']}
        collection.insert(data)
        return item

Overwriting dailyhotel/dailyhotel/pipelines.py


In [9]:
# 8. settings.py -> pipeline 설정변경

```
ITEM_PIPELINES = {
   'dailyhotel.pipelines.DailyhotelPipeline': 300,
}
```

In [10]:
# 9. 프로젝트 실행!

In [9]:
!/bin/bash run.sh

2021-03-12 15:20:09 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: dailyhotel)
2021-03-12 15:20:09 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Jan 15 2021, 05:57:06) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1i  8 Dec 2020), cryptography 3.3.1, Platform Linux-5.4.0-1038-aws-x86_64-with-debian-buster-sid
2021-03-12 15:20:09 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-12 15:20:09 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'dailyhotel',
 'NEWSPIDER_MODULE': 'dailyhotel.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['dailyhotel.spiders']}
2021-03-12 15:20:09 [scrapy.extensions.telnet] INFO: Telnet Password: 0f46c2bc19e50f19
2021-03-12 15:20:09 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsa

2021-03-12 15:20:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/30202?stays=1&dateCheckIn=2021-03-12&regionStayType=all> (referer: https://www.dailyhotel.com/)
2021-03-12 15:20:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/28591?stays=1&dateCheckIn=2021-03-12&regionStayType=all>
{'date': '2021-03-12',
 'level': '특2급',
 'link': 'https://www.dailyhotel.com/stays/28591?dateCheckIn=2021-03-12&stays=1',
 'location': '서울 강남구 테헤란로 415',
 'name': 'L7 강남 바이 롯데',
 'platform': '데일리호텔',
 'price': 106900,
 'review_count': 1233,
 'room_type': '[레이트체크인 18시 체크인] STANDARD ',
 'score': 4.5}
2021-03-12 15:20:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/28591?stays=1&dateCheckIn=2021-03-12&regionStayType=all>
{'date': '2021-03-12',
 'level': '특2급',
 'link': 'https://www.dailyhotel.com/stays/28591?dateCheckIn=2021-03-12&

2021-03-12 15:20:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/789?stays=1&dateCheckIn=2021-03-12&regionStayType=all>
{'date': '2021-03-12',
 'level': '특2급',
 'link': 'https://www.dailyhotel.com/stays/789?dateCheckIn=2021-03-12&stays=1',
 'location': '서울 영등포구 의사당대로 16',
 'name': '글래드 여의도',
 'platform': '데일리호텔',
 'price': 100000,
 'review_count': 4380,
 'room_type': '[Room only] 점보 트윈 (패밀리 트윈) ',
 'score': 4.5}
2021-03-12 15:20:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/789?stays=1&dateCheckIn=2021-03-12&regionStayType=all>
{'date': '2021-03-12',
 'level': '특2급',
 'link': 'https://www.dailyhotel.com/stays/789?dateCheckIn=2021-03-12&stays=1',
 'location': '서울 영등포구 의사당대로 16',
 'name': '글래드 여의도',
 'platform': '데일리호텔',
 'price': 125500,
 'review_count': 4380,
 'room_type': '[Room only] 디럭스 더블 ',
 'score': 4.5}
2021-03-12 15:20:15 [scrapy.core.engine] DEBUG: Crawle

2021-03-12 15:20:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/801?stays=1&dateCheckIn=2021-03-12&regionStayType=all>
{'date': '2021-03-12',
 'level': '특2급',
 'link': 'https://www.dailyhotel.com/stays/801?dateCheckIn=2021-03-12&stays=1',
 'location': '서울 강남구 영동대로 737',
 'name': '호텔 리베라 청담',
 'platform': '데일리호텔',
 'price': 115000,
 'review_count': 3229,
 'room_type': '[룸서비스 도시락 PKG] 럭셔리 ',
 'score': 4.1}
2021-03-12 15:20:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/801?stays=1&dateCheckIn=2021-03-12&regionStayType=all>
{'date': '2021-03-12',
 'level': '특2급',
 'link': 'https://www.dailyhotel.com/stays/801?dateCheckIn=2021-03-12&stays=1',
 'location': '서울 강남구 영동대로 737',
 'name': '호텔 리베라 청담',
 'platform': '데일리호텔',
 'price': 115000,
 'review_count': 3229,
 'room_type': '[룸서비스 도시락 PKG] 럭셔리 ',
 'score': 4.1}
2021-03-12 15:20:15 [scrapy.core.scraper] DEBUG: Scraped fro

2021-03-12 15:20:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/6933?stays=1&dateCheckIn=2021-03-12&regionStayType=all>
{'date': '2021-03-12',
 'level': '특1급',
 'link': 'https://www.dailyhotel.com/stays/6933?dateCheckIn=2021-03-12&stays=1',
 'location': '서울 중구 동호로 249',
 'name': '서울신라호텔',
 'platform': '데일리호텔',
 'price': 544500,
 'review_count': 2492,
 'room_type': '[The Plus PKG] - Business Deluxe ',
 'score': 4.9}
2021-03-12 15:20:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/6933?stays=1&dateCheckIn=2021-03-12&regionStayType=all>
{'date': '2021-03-12',
 'level': '특1급',
 'link': 'https://www.dailyhotel.com/stays/6933?dateCheckIn=2021-03-12&stays=1',
 'location': '서울 중구 동호로 249',
 'name': '서울신라호텔',
 'platform': '데일리호텔',
 'price': 556600,
 'review_count': 2492,
 'room_type': 'The Deluxe - Grand Corner Deluxe ',
 'score': 4.9}
2021-03-12 15:20:15 [scrapy.core.scrap

...

2021-03-12 15:20:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/647?stays=1&dateCheckIn=2021-03-12&regionStayType=all>
{'date': '2021-03-12',
 'level': '1급',
 'link': 'https://www.dailyhotel.com/stays/647?dateCheckIn=2021-03-12&stays=1',
 'location': '서울 중구 남대문로 78',
 'name': '이비스 앰배서더 명동',
 'platform': '데일리호텔',
 'price': 143000,
 'review_count': 347,
 'room_type': '[봄맞이 러브썸 PKG] 스탠다드 ',
 'score': 4.2}
2021-03-12 15:20:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/602?stays=1&dateCheckIn=2021-03-12&regionStayType=all>
{'date': '2021-03-12',
 'level': '1급',
 'link': 'https://www.dailyhotel.com/stays/602?dateCheckIn=2021-03-12&stays=1',
 'location': '서울 구로구 디지털로 300',
 'name': '롯데시티호텔 구로',
 'platform': '데일리호텔',
 'price': 73200,
 'review_count': 5264,
 'room_type': '[Room only] 스탠다드 ',
 'score': 4.7}
2021-03-12 15:20:22 [scrapy.core.scraper] DEBUG: Scraped from <200

2021-03-12 15:20:22 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/32199?stays=1&dateCheckIn=2021-03-12&regionStayType=all> (referer: https://www.dailyhotel.com/)
2021-03-12 15:20:22 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/27914?stays=1&dateCheckIn=2021-03-12&regionStayType=all> (referer: https://www.dailyhotel.com/)
2021-03-12 15:20:22 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/27910?stays=1&dateCheckIn=2021-03-12&regionStayType=all> (referer: https://www.dailyhotel.com/)
2021-03-12 15:20:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/35556?stays=1&dateCheckIn=2021-03-12&regionStayType=all>
{'date': '2021-03-12',
 'level': '특2급',
 'link': 'https://www.dailyhotel.com/stays/35556?dateCheckIn=2021-03-12&stays=1',
 'location': '서울특별시 중구 칠패로 27

2021-03-12 15:20:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/29362?stays=1&dateCheckIn=2021-03-12&regionStayType=all>
{'date': '2021-03-12',
 'level': '1급',
 'link': 'https://www.dailyhotel.com/stays/29362?dateCheckIn=2021-03-12&stays=1',
 'location': '서울 중구 동호로 354',
 'name': '라마다 서울 동대문',
 'platform': '데일리호텔',
 'price': 90000,
 'review_count': 997,
 'room_type': '디럭스 프라임 킹 ',
 'score': 4.5}
2021-03-12 15:20:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/29362?stays=1&dateCheckIn=2021-03-12&regionStayType=all>
{'date': '2021-03-12',
 'level': '1급',
 'link': 'https://www.dailyhotel.com/stays/29362?dateCheckIn=2021-03-12&stays=1',
 'location': '서울 중구 동호로 354',
 'name': '라마다 서울 동대문',
 'platform': '데일리호텔',
 'price': 90000,
 'review_count': 997,
 'room_type': '패밀리트윈 ',
 'score': 4.5}
2021-03-12 15:20:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dai

2021-03-12 15:20:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/27910?stays=1&dateCheckIn=2021-03-12&regionStayType=all>
{'date': '2021-03-12',
 'level': '특1급',
 'link': 'https://www.dailyhotel.com/stays/27910?dateCheckIn=2021-03-12&stays=1',
 'location': '서울 용산구 청파로20길 95',
 'name': '그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산',
 'platform': '데일리호텔',
 'price': 264600,
 'review_count': 745,
 'room_type': '[HOT DEAL!|조식 2인 포함] 주니어 스위트 ',
 'score': 4.8}
2021-03-12 15:20:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.dailyhotel.com/newdelhi/goodnight/api/v9/hotel/27910?stays=1&dateCheckIn=2021-03-12&regionStayType=all>
{'date': '2021-03-12',
 'level': '특1급',
 'link': 'https://www.dailyhotel.com/stays/27910?dateCheckIn=2021-03-12&stays=1',
 'location': '서울 용산구 청파로20길 95',
 'name': '그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산',
 'platform': '데일리호텔',
 'price': 269000,
 'review_count': 745,
 'room_type': '[HOT DEAL!] 슈페리어 스위트 ',
 'score': 4.8}
20

2021-03-12 15:20:22 [scrapy.core.engine] INFO: Closing spider (finished)
2021-03-12 15:20:22 [scrapy.extensions.feedexport] INFO: Stored csv feed (1807 items) in: dailyhotel.csv
2021-03-12 15:20:22 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 107616,
 'downloader/request_count': 240,
 'downloader/request_method_count/GET': 240,
 'downloader/response_bytes': 7411850,
 'downloader/response_count': 240,
 'downloader/response_status_count/200': 240,
 'elapsed_time_seconds': 13.561206,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 3, 12, 6, 20, 22, 898321),
 'item_scraped_count': 1807,
 'log_count/DEBUG': 2065,
 'log_count/ERROR': 5,
 'log_count/INFO': 11,
 'memusage/max': 61100032,
 'memusage/startup': 61100032,
 'request_depth_max': 1,
 'response_received_count': 240,
 'robotstxt/request_count': 1,
 'robotstxt/response_count': 1,
 'robotstxt/response_status_count/200': 1,
 'scheduler/dequeued': 239,
 'scheduler/dequeued/memory':

In [8]:
# 12초

In [10]:
import pandas as pd
data = pd.read_csv('dailyhotel/dailyhotel.csv', encoding='utf-8-sig')
print(data.shape)
data.tail()

(1807, 10)


,date,level,link,location,name,platform,price,review_count,room_type,score
1802,2021-03-12,특1급,https://www.dailyhotel.com/stays/27910?dateChe...,서울 용산구 청파로20길 95,그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산,데일리호텔,445000,745,[HOT DEAL!] 디럭스 스위트 2베드룸,4.8
1803,2021-03-12,특1급,https://www.dailyhotel.com/stays/27910?dateChe...,서울 용산구 청파로20길 95,그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산,데일리호텔,467500,745,디럭스 스위트 2베드룸,4.8
1804,2021-03-12,특1급,https://www.dailyhotel.com/stays/27910?dateChe...,서울 용산구 청파로20길 95,그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산,데일리호텔,506600,745,[HOT DEAL!ㅣ조식 2인 포함] 디럭스 스위트 2베드룸,4.8
1805,2021-03-12,특1급,https://www.dailyhotel.com/stays/27910?dateChe...,서울 용산구 청파로20길 95,그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산,데일리호텔,529100,745,[조식 PKG] 디럭스 스위트 2베드룸,4.8
1806,2021-03-12,특1급,https://www.dailyhotel.com/stays/27910?dateChe...,서울 용산구 청파로20길 95,그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산,데일리호텔,577000,745,[The Lounge l EFL 2인 포함PKG] 디럭스 스위트 2베드룸,4.8


In [ ]:
# 10. crontab 실행